<a href="https://colab.research.google.com/github/GrishaRybolovel/TestTask/blob/main/GrigoriiRybolovlev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Для начала выгрузим файл в переменную
with open('trial_task.json') as f:
    json_data = json.load(f)

# Затем нормализуем наши данные в табличку
df = pd.json_normalize(json_data, record_path='products', meta=['order_id', 'warehouse_name', 'highway_cost'])

print(df)

                      product  price  quantity order_id       warehouse_name  \
0       ломтик июльского неба    450         1    11973               Мордор   
1             билет в Израиль   1000         3    11973               Мордор   
2            статуэтка Ленина    200         3    11973               Мордор   
3             билет в Израиль   1000         1    62239  хутор близ Диканьки   
4           зеленая пластинка     10         2    85794           отель Лето   
..                        ...    ...       ...      ...                  ...   
192  автограф Стаса Барецкого    600         1    79293           отель Лето   
193     ломтик июльского неба    450         1    79293           отель Лето   
194                   плюмбус    250         2     2930               Мордор   
195                   плюмбус    250         1     2930               Мордор   
196                   плюмбус    250         3      124  хутор близ Диканьки   

    highway_cost  
0            -70  
1

In [9]:
# 1 задание
# Находим тарифы стоимости для каждого склада
df_unique = df[['warehouse_name', 'highway_cost']].drop_duplicates()

print(df_unique)

          warehouse_name highway_cost
0                 Мордор          -70
3    хутор близ Диканьки          -15
4             отель Лето          -50
5                 Мордор          -30
7             отель Лето          -75
10      остров невезения          -15
11            гиперборея         -160
14            гиперборея          -80
17                Мордор          -20
18   хутор близ Диканьки         -105
21            отель Лето          -25
23   хутор близ Диканьки          -60
28            гиперборея          -20
29      остров невезения          -35
32                Мордор          -10
33      остров невезения          -10
34   хутор близ Диканьки          -75
42   хутор близ Диканьки          -90
45            отель Лето         -175
52      остров невезения          -20
54      остров невезения          -30
57   хутор близ Диканьки          -45
60   хутор близ Диканьки         -120
63   хутор близ Диканьки          -30
71            гиперборея          -40
75          

In [10]:
# 2 задание
# Конвертируем поля 'price', 'quantity' и 'highway_cost' в integer
df[['price', 'quantity', 'highway_cost']] = df[['price', 'quantity', 'highway_cost']].apply(pd.to_numeric)

# Считаем 'income', 'expenses', and 'profit' для каждого продукта
df['income'] = df['price'] * df['quantity']
df['expenses'] = df['highway_cost'] * df['quantity']
df['profit'] = df['income'] - df['expenses']

# Группируем по полю 'product' и считаем итоговые поля 'quantity', 'income', 'expenses' и 'profit' для каждого продукта
df_grouped = df.groupby('product').agg({
    'quantity': 'sum',
    'income': 'sum',
    'expenses': 'sum',
    'profit': 'sum'
}).reset_index()

print(df_grouped)

                    product  quantity  income  expenses  profit
0  автограф Стаса Барецкого        48   28800     -3930   32730
1           билет в Израиль        58   58000     -5110   63110
2         зеленая пластинка        61     610     -4595    5205
3     ломтик июльского неба        42   18900     -2495   21395
4                   плюмбус        65   16250     -4655   20905
5    подписка на suppi-блог        33    4950     -2670    7620
6          статуэтка Ленина        68   13600     -4565   18165


In [11]:
# 3 задание
# Считаем поле profit для каждого
df_orders = df.groupby('order_id').agg({
    'profit': 'sum'
}).reset_index()

# Переименовываем 'profit' в 'order_profit'
df_orders.rename(columns={'profit': 'order_profit'}, inplace=True)

print(df_orders)

# Считаем среднее значение поля profit
avg_profit = df_orders['order_profit'].mean()
print('Average Order Profit: ', avg_profit)

    order_id  order_profit
0        124           885
1       1391           520
2       2091          1800
3       2108           230
4       2558           805
..       ...           ...
95     98100          1620
96     98423          1260
97     99220          1525
98     99246          1935
99     99324          1620

[100 rows x 2 columns]
Average Order Profit:  1691.3


In [12]:
# 4 задание
# Группируем по обоим полям 'warehouse_name' и 'product'
df_product_profit = df.groupby(['warehouse_name', 'product']).agg({
    'quantity': 'sum',
    'profit': 'sum'
}).reset_index()

# Группируем по полю 'warehouse_name' чтобы получить итоговое значение поля profit для каждого склада
df_warehouse_profit = df.groupby('warehouse_name').agg({
    'profit': 'sum'
}).reset_index()

# Сливаем два датасета по полю 'warehouse_name'
df_merged = pd.merge(df_product_profit, df_warehouse_profit, on='warehouse_name', suffixes=('_product', '_warehouse'))

# Считаем процент прибыли от продукта к складской прибыли
df_merged['percent_profit_product_of_warehouse'] = (df_merged['profit_product'] / df_merged['profit_warehouse']) * 100

print(df_merged[['warehouse_name', 'product', 'quantity', 'profit_product', 'percent_profit_product_of_warehouse']])

         warehouse_name                   product  quantity  profit_product  \
0                Мордор  автограф Стаса Барецкого         4            2670   
1                Мордор           билет в Израиль         9            9350   
2                Мордор         зеленая пластинка        11             520   
3                Мордор     ломтик июльского неба         3            1460   
4                Мордор                   плюмбус         6            1830   
5                Мордор    подписка на suppi-блог         8            1700   
6                Мордор          статуэтка Ленина        10            2440   
7            гиперборея  автограф Стаса Барецкого        12            8500   
8            гиперборея           билет в Израиль        21           23000   
9            гиперборея         зеленая пластинка        10            1100   
10           гиперборея     ломтик июльского неба        13            6970   
11           гиперборея                   плюмбус   

In [13]:
# 5 задание
# Сортируем данные по полю 'percent_profit_product_of_warehouse' в убывающем порядке
df_sorted = df_merged.sort_values('percent_profit_product_of_warehouse', ascending=False)

# Считаем накопленный процент
df_sorted['accumulated_percent_profit_product_of_warehouse'] = df_sorted['percent_profit_product_of_warehouse'].cumsum()


print(df_sorted[['warehouse_name', 'product', 'quantity', 'profit_product', 'percent_profit_product_of_warehouse', 'accumulated_percent_profit_product_of_warehouse']])

         warehouse_name                   product  quantity  profit_product  \
21           отель Лето           билет в Израиль        15           16925   
8            гиперборея           билет в Израиль        21           23000   
1                Мордор           билет в Израиль         9            9350   
27  хутор близ Диканьки  автограф Стаса Барецкого        22           14715   
17     остров невезения     ломтик июльского неба        10            4770   
19     остров невезения          статуэтка Ленина        18            4015   
28  хутор близ Диканьки           билет в Израиль        10           10810   
18     остров невезения                   плюмбус        14            3825   
31  хутор близ Диканьки                   плюмбус        27            8865   
15     остров невезения           билет в Израиль         3            3025   
7            гиперборея  автограф Стаса Барецкого        12            8500   
20           отель Лето  автограф Стаса Барецкого   

In [14]:
# 6 задание
# Определяем границы категорий
bins = [0, 70, 90, df_sorted['accumulated_percent_profit_product_of_warehouse'].max()]
labels = ['A', 'B', 'C']

# Записываем границы на основе 'accumulated_percent_profit_product_of_warehouse'
df_sorted['category'] = pd.cut(df_sorted['accumulated_percent_profit_product_of_warehouse'], bins=bins, labels=labels, include_lowest=True)

print(df_sorted[['warehouse_name', 'product', 'quantity', 'profit_product', 'percent_profit_product_of_warehouse', 'accumulated_percent_profit_product_of_warehouse', 'category']])

         warehouse_name                   product  quantity  profit_product  \
21           отель Лето           билет в Израиль        15           16925   
8            гиперборея           билет в Израиль        21           23000   
1                Мордор           билет в Израиль         9            9350   
27  хутор близ Диканьки  автограф Стаса Барецкого        22           14715   
17     остров невезения     ломтик июльского неба        10            4770   
19     остров невезения          статуэтка Ленина        18            4015   
28  хутор близ Диканьки           билет в Израиль        10           10810   
18     остров невезения                   плюмбус        14            3825   
31  хутор близ Диканьки                   плюмбус        27            8865   
15     остров невезения           билет в Израиль         3            3025   
7            гиперборея  автограф Стаса Барецкого        12            8500   
20           отель Лето  автограф Стаса Барецкого   